# Part I — Foundations  
## 3. Developer Tooling and Workflow (Industry Standard)

Django skill in real teams is not just “can I build a view.” It’s also:

- Can you reproduce a clean environment?
- Can you collaborate safely with Git?
- Can you enforce consistent formatting and catch bugs early?
- Can you run tests locally and in CI?
- Can you debug fast using logs, breakpoints, and profiling tools?

This chapter sets up a professional baseline you will carry through the rest of the
workbook.

---

## 3.0 Learning Outcomes (What you must be able to do)

By the end of this chapter, you should be able to:

1. Initialize a repo with a professional file structure and a correct `.gitignore`.
2. Use Git confidently:
   - stage/commit
   - branch
   - merge vs rebase (and when)
   - resolve basic conflicts
   - write high-quality commit messages
3. Set up automated code quality checks:
   - formatter (consistent code style)
   - linter (bug/quality checks)
   - optional type checks
4. Enforce standards automatically with `pre-commit`.
5. Run tests locally in a repeatable way (and understand how CI will run them).
6. Debug using:
   - breakpoints (`breakpoint()`, `pdb`)
   - logging
   - IDE debugging
7. Know how to read docs like a professional (fast, accurate, with minimal guessing).

---

## 3.1 The “Professional Django” Workflow (Mental Model)

A typical industry loop looks like this:

1. Create a feature branch
2. Write code (with formatter/linter)
3. Run tests locally
4. Commit with a meaningful message
5. Push, open PR
6. CI runs: lint + tests + (sometimes) type checks + security checks
7. Code review
8. Merge to main (or merge to trunk)
9. Deploy (later chapters)

The key idea: **automation prevents mistakes from reaching production**.

---

## 3.2 Git Fundamentals (What You Actually Need Daily)

### 3.2.1 Why Git is non-negotiable
Git provides:
- history (who changed what and why)
- collaboration (multiple people in parallel)
- safe experimentation (branches)
- rollback (revert a bad change)
- code review workflows (PRs/MRs)

### 3.2.2 The 3 “areas” of Git (this removes 80% of confusion)

Git is easiest when you understand these areas:

1. **Working directory**: your files on disk (edited or not)
2. **Staging area (index)**: what will go into the next commit
3. **Repository (commits)**: permanent snapshots with history

Typical flow:

```text
edit file -> stage -> commit -> push
```

### 3.2.3 Commands you will use constantly

#### Initialize a repo
```bash
git init
```

#### See status
```bash
git status
```

#### Stage files
Stage one file:
```bash
git add README.md
```

Stage everything changed:
```bash
git add -A
```

#### Commit
```bash
git commit -m "Add project README"
```

#### View history
```bash
git log --oneline --decorate --graph --all
```

### 3.2.4 Branching: the simplest safe workflow

Create a branch:
```bash
git checkout -b feature/user-auth
```

(Modern git also supports:)
```bash
git switch -c feature/user-auth
```

Switch back to main:
```bash
git checkout main
```

### 3.2.5 Merge vs rebase (with practical team guidance)

#### Merge
A merge combines branches and preserves their history.

- Pros: safest, preserves exact history, easiest for beginners
- Cons: can create “merge commits” that clutter history

#### Rebase
A rebase replays your commits on top of another base.

- Pros: linear history, nice readable logs
- Cons: can be dangerous if you rebase commits that others already pulled

**Industry rule (safe practice):**
- Rebase your *local feature branch* onto updated main to resolve conflicts early.
- Avoid rebasing shared/public branches after pushing (unless your team explicitly
  uses that workflow).

### 3.2.6 Pull strategies (avoid accidental messy merges)

To pull changes from origin:

```bash
git pull
```

But “how pull behaves” depends on configuration (merge vs rebase).

A common, clean setup for many teams:

```bash
git config --global pull.rebase true
```

This makes `git pull` do “fetch + rebase” by default.

If you’re not comfortable yet, it’s okay to keep default merge pulls—just be
consistent and understand what it does.

### 3.2.7 Commit messages: what “good” looks like

Good commit messages answer:
- **What changed?**
- **Why?**

Bad:
- “fix”
- “changes”
- “update stuff”

Good:
- “Add linting and formatting via pre-commit”
- “Validate page_size query param and cap at 100”
- “Fix N+1 query in article list by adding prefetch_related”

Many teams use **Conventional Commits** (optional but common):

```text
feat: add pagination to article list
fix: handle invalid page parameter
chore: configure pre-commit hooks
refactor: extract pricing logic into service module
test: add tests for slugify
```

This improves release notes, automation, and clarity.

---

## 3.3 Repo Hygiene (Files That Make Teams Productive)

### 3.3.1 `.gitignore` (stop committing junk)
For Python/Django, you typically ignore:
- virtual env folders (`.venv/`)
- Python bytecode (`__pycache__/`, `*.pyc`)
- environment files (`.env`)
- local DB files (like `db.sqlite3` for dev, depending on team)
- editor folders (`.vscode/`, `.idea/`) (team policy varies)
- test/cache artifacts (`.pytest_cache/`, `.mypy_cache/`, `.ruff_cache/`)

A solid starter `.gitignore`:

```gitignore
# Python
__pycache__/
*.py[cod]
*.pyd
*.pyo

# Virtual environments
.venv/
venv/
ENV/
env/

# Django
*.log
local_settings.py
db.sqlite3
db.sqlite3-journal
media/

# Env vars
.env
.env.*

# Tool caches
.pytest_cache/
.mypy_cache/
.ruff_cache/

# OS / Editor
.DS_Store
.idea/
.vscode/
```

**Important note:** ignoring `db.sqlite3` is common because it’s local dev data.
Some teams commit it for tutorials; most professional apps do not.

### 3.3.2 `README.md` (professional baseline)
A good README includes:
- what the project is
- how to set up dev
- how to run tests
- how to run lint/format
- how to run the server
- how to configure env vars

You’ll keep expanding this as the workbook progresses.

### 3.3.3 `.editorconfig` (prevents whitespace wars)
This helps unify indentation, line endings, trailing spaces across editors.

Create `.editorconfig`:

```ini
root = true

[*]
charset = utf-8
end_of_line = lf
insert_final_newline = true
trim_trailing_whitespace = true

[*.py]
indent_style = space
indent_size = 4

[*.md]
trim_trailing_whitespace = false
```

Why this matters:
- avoids constant “only whitespace changed” diffs
- makes code review cleaner

---

## 3.4 Formatter vs Linter vs Type Checker (Don’t Mix These Up)

### 3.4.1 Formatter (example: Black)
A formatter rewrites code to a consistent style.

- Purpose: eliminate debates about spacing/line breaks
- Output: reformatted code
- Benefit: code review focuses on logic, not style

### 3.4.2 Linter (example: Ruff)
A linter finds:
- bugs (unused imports, unreachable code)
- suspicious patterns
- complexity
- style issues (depending on configuration)

Some linters can auto-fix issues (imports, simple problems).

### 3.4.3 Type checker (example: mypy/pyright)
A type checker verifies your hints and catches:
- wrong argument types
- missing attributes
- incorrect return types

Type checks can be introduced gradually.

**Industry reality:** many Django teams start with:
- formatter + linter + tests
and add typing later as needed.

---

## 3.5 Recommended Baseline Toolchain (Simple, Strong, Common)

A widely accepted baseline today:

- **Black**: formatting
- **Ruff**: linting (and often import sorting)
- **pre-commit**: enforce checks before commits
- **pytest** (later with pytest-django): tests

We’ll set up these now (without needing a Django project yet).

---

## 3.6 Project Setup (Tooling-First Skeleton You’ll Reuse)

Create a folder for your main project (if you don’t already have one):

```bash
mkdir django-mastery
cd django-mastery
```

Initialize Git:

```bash
git init
```

Create these files:

```text
django-mastery/
  .gitignore
  .editorconfig
  README.md
  requirements.txt
  requirements-dev.txt
  pyproject.toml
  .pre-commit-config.yaml
```

### 3.6.1 `requirements-dev.txt`
This is for developer-only tooling (not production runtime).

Example:

```text
black
ruff
pre-commit
pytest
```

Install dev tools:

```bash
python -m pip install -r requirements-dev.txt
```

---

## 3.7 Configure Black + Ruff in `pyproject.toml` (With Explanation)

Create `pyproject.toml`:

```toml
[tool.black]
line-length = 88
target-version = ["py311"]
include = '\.pyi?$'

[tool.ruff]
line-length = 88
target-version = "py311"

[tool.ruff.lint]
select = [
  "E",   # pycodestyle errors
  "F",   # pyflakes
  "I",   # import sorting
  "B",   # flake8-bugbear
  "UP",  # pyupgrade
]
ignore = []

[tool.ruff.lint.per-file-ignores]
"__init__.py" = ["F401"]
```

### Explain every part (why this is structured like this)

#### Black section
- `line-length = 88`: Black’s default style. This is not magic—just a widely used
  compromise between readability and fitting on screens.
- `target-version`: helps Black format code appropriately for your Python version.
  Use the Python version your project will run in production.

#### Ruff section
- `line-length`: align with formatter to reduce conflicts.
- `target-version`: helps Ruff suggest modern Python patterns only if your runtime
  supports them.

#### Ruff rule selection
- `"E"` and `"F"`: baseline bug/quality checks (very common).
- `"I"`: enforces consistent import ordering (keeps diffs clean).
- `"B"`: catches real bugs (like mutable default arguments, suspicious comparisons).
- `"UP"`: encourages modern Python syntax improvements.

#### Per-file ignore for `__init__.py`
`__init__.py` often contains re-exports:

```python
from .something import Thing
```

Ruff might call that “unused import” (`F401`) even though it’s intentional. That’s
why we ignore `F401` in `__init__.py`.

---

## 3.8 Running Formatter and Linter (What they do, and what output means)

### 3.8.1 Black (formatter)
Run:

```bash
python -m black .
```

What happens:
- Black reformats Python files in place
- If nothing changes, it reports “left unchanged”

### 3.8.2 Ruff (linter)
Check only:

```bash
python -m ruff check .
```

Auto-fix what it safely can:

```bash
python -m ruff check . --fix
```

Important:
- A linter is not always safe to auto-fix everything (depends on rule).
- Ruff generally auto-fixes safe transformations (like import sorting).

---

## 3.9 Pre-commit (Enforce Standards Automatically)

### 3.9.1 Why pre-commit is industry standard
Without pre-commit:
- people “forget” to format
- CI fails later
- PRs are noisy
- style issues waste reviewer time

With pre-commit:
- checks run before you commit
- code stays consistently formatted
- fewer CI failures

### 3.9.2 Create `.pre-commit-config.yaml`
Create:

```yaml
repos:
  - repo: https://github.com/pre-commit/pre-commit-hooks
    rev: v4.6.0
    hooks:
      - id: end-of-file-fixer
      - id: trailing-whitespace
      - id: check-yaml
      - id: check-added-large-files

  - repo: https://github.com/psf/black
    rev: 24.8.0
    hooks:
      - id: black
        language_version: python3

  - repo: https://github.com/astral-sh/ruff-pre-commit
    rev: v0.6.9
    hooks:
      - id: ruff
        args: ["--fix"]
```

### Explain what each hook does (why you want it)

- `end-of-file-fixer`: ensures files end with a newline (POSIX convention; reduces
  weird diffs).
- `trailing-whitespace`: removes trailing spaces (commonly accidental).
- `check-yaml`: prevents broken YAML files (CI configs, etc).
- `check-added-large-files`: stops you from committing huge files accidentally.
- `black`: formats Python.
- `ruff`: lints and auto-fixes safe issues (imports etc).

### 3.9.3 Install and run pre-commit
Install git hooks:

```bash
pre-commit install
```

Run on all files (first time):

```bash
pre-commit run --all-files
```

What you’ll see:
- Each hook runs and either “Passed” or “Failed”
- If it fails and modifies files, you re-stage and commit again

This is normal and expected.

---

## 3.10 Concrete Example: Create Bad Code, Then Watch Tools Fix It

Create a file `example.py`:

```python
import os, sys


def add(a,b):
  return a+ b
```

Now run:

```bash
python -m ruff check . --fix
python -m black .
```

### What happens and why

- Ruff will likely:
  - split imports into separate lines
  - sort imports
  - complain about unused imports (depending on usage)

- Black will:
  - fix indentation
  - normalize spaces
  - format function signature spacing

After formatting, you should end up with something like:

```python
import os
import sys


def add(a, b):
    return a + b
```

Even if this seems “cosmetic,” it matters because:
- diffs become predictable
- merge conflicts drop
- reviews focus on logic

---

## 3.11 Debugging (Fast Feedback Techniques You’ll Use Daily)

### 3.11.1 `breakpoint()` (the modern, simple debugger entry)
In Python, you can pause execution:

```python
def compute_total(items):
    breakpoint()
    return sum(item["price"] for item in items)
```

Run the code, and Python drops into an interactive debugger prompt.

Why this is powerful:
- inspect variables
- step through execution
- confirm assumptions instantly

### 3.11.2 `pdb` basics (what to type when paused)
When stopped at a breakpoint, common commands:
- `l` (list code)
- `n` (next line)
- `s` (step into)
- `c` (continue)
- `p variable` (print)
- `q` (quit)

This will later be extremely useful in Django views, forms, and management commands.

### 3.11.3 Logging vs debugging: when to use which
- Use breakpoints when developing locally and you need to explore state.
- Use logging when you need visibility in production or in CI runs.

A professional Django developer uses both.

---

## 3.12 Testing Workflow (Baseline Now, Django Later)

We’ll set up `pytest` now so you have the habit. Later, we’ll add `pytest-django`
and database-backed tests.

### 3.12.1 Why tests are not optional in industry
Tests:
- prevent regressions
- document behavior
- speed up refactoring
- make PRs safer

### 3.12.2 Minimal pytest setup
Create `tests/test_math.py`:

```python
def add(a: int, b: int) -> int:
    return a + b


def test_addition():
    assert add(2, 3) == 5
```

Run:

```bash
python -m pytest
```

Even this tiny test matters because it proves:
- your environment is correct
- test execution is consistent
- you can extend this approach into Django tests later

---

## 3.13 CI Mindset (Even Before You Have CI)

Even if you haven’t set up GitHub Actions/GitLab CI yet, you should behave as if you
have, because you will.

A “CI-like local run” is typically:

```bash
python -m ruff check .
python -m black . --check
python -m pytest
```

Explanation:
- `black --check` fails if formatting would change (CI shouldn’t reformat for you)
- linter fails on violations
- tests must pass

Later, we’ll automate this in CI.

---

## 3.14 Docs Reading Strategy (How Professionals Learn Django Fast)

### 3.14.1 Don’t read docs linearly
Instead:
1. Start with the official tutorial to build a mental model.
2. When you hit a concept (like middleware), read that specific section.
3. Use the reference docs when you already know what you’re looking for.

### 3.14.2 Use “why + example + API reference”
For any Django feature:
- Find “why it exists”
- Find a minimal working example
- Then read the API reference (methods/options)

This prevents shallow memorization.

### 3.14.3 Keep a “glossary notebook”
Write down:
- request vs response
- middleware
- context processors
- migrations
- queryset evaluation

This reduces confusion later when terms stack.

---

# 3.15 Hands-On Lab (Do This Now): Tooling Baseline Commit

This lab produces a clean first commit that you’ll build on.

## Step 1 — Create essential files
Create:
- `.gitignore`
- `.editorconfig`
- `pyproject.toml`
- `requirements-dev.txt`
- `.pre-commit-config.yaml`
- `README.md`

(Use the contents provided above.)

## Step 2 — Install dev tools
```bash
python -m pip install -r requirements-dev.txt
```

## Step 3 — Install pre-commit hooks
```bash
pre-commit install
pre-commit run --all-files
```

## Step 4 — Add a quick test
Create `tests/test_math.py` and run:

```bash
python -m pytest
```

## Step 5 — Make your first professional commit
```bash
git add -A
git commit -m "chore: set up repo tooling (ruff, black, pre-commit, pytest)"
```

What you now have:
- repeatable formatting/linting
- automatic checks on commit
- a test harness you’ll extend into Django tests

---

## 3.16 Exercises (to confirm mastery)

1. Explain the difference between:
   - formatting and linting
   - linting and type checking
2. Why should CI run `black --check` instead of `black`?
3. What happens if a pre-commit hook modifies files? What must you do next?
4. Create a branch, make a commit, then merge it back into main.
   - Write down the exact commands you used.

---

## 3.17 Checklist (You’re ready to proceed if all are true)

- [ ] You can initialize Git and commit cleanly.
- [ ] You can run Ruff and Black and explain what they do.
- [ ] You have pre-commit installed and hooks run on commit.
- [ ] You can run pytest locally.
- [ ] You understand how these map to CI later.

---